In [1]:
import argparse
import os
import sys
from os import mkdir

import torch.nn.functional as F

sys.path.append('.')
from config import cfg

from data.build import SAMDataLoader
from modeling.segment_anything import prepare_sam

from utils.logger import setup_logger

from config import cfg  # Import the default config file
import gc

from tqdm import tqdm
import torch

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from modeling.segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from torch.nn.functional import threshold, normalize

from utils.data_support import get_bbox_point_and_inputlabel_prompts

from engine.sam_trainer import SAMTrainer
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import RichProgressBar
from lightning.pytorch.loggers import WandbLogger

In [2]:
cfg.freeze()

In [3]:
model = prepare_sam(checkpoint=cfg.MODEL.CHECKPOINT, model_type = 'base')
device = cfg.MODEL.DEVICE
epochs = cfg.SOLVER.MAX_EPOCHS

In [4]:
for name, param in model.named_parameters():
    if name.startswith('image_encoder') or name.startswith('prompt_encoder'):
        param.requires_grad = False

In [5]:
sam_datamodule = SAMDataLoader(cfg)
# wandb_logger = WandbLogger(project='SAM')

Output()

[02:42:21] Data loaded! Total number of images: 1000                                             ]8;id=717234;file:///Users/mx98/workspace/hannah-rae/SAM-FineTuning/data/datasets/planetscope.py\planetscope.py]8;;\:]8;id=899302;file:///Users/mx98/workspace/hannah-rae/SAM-FineTuning/data/datasets/planetscope.py#196\196]8;;\

In [6]:
trainer = Trainer(
    callbacks=[RichProgressBar()], 
    # logger=wandb_logger, 
    max_epochs=epochs)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
modelx = SAMTrainer(model)

In [9]:
trainer.fit(modelx, sam_datamodule)
# sam_datamodule.setup('asd')
# sample = sam_datamodule.train_dataset[0]

┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name       ┃ Type      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model      │ Sam       │ 93.7 M │
│ 1 │ Focal_Loss │ FocalLoss │      0 │
│ 2 │ Dice_Loss  │ DiceLoss  │      0 │
│ 3 │ Iou_Loss   │ IoULoss   │      0 │
└───┴────────────┴───────────┴────────┘

Trainable params: 4.1 M                                                                                            
Non-trainable params: 89.7 M                                                                                       
Total params: 93.7 M                                                                                               
Total estimated model params size (MB): 374

Output()